In [1]:
from matplotlib import pyplot as plt

import numpy as np
np.random.seed(42)
import random
random.seed(42)

In [2]:
from matplotlib import pyplot as plt
import numpy as np
#plt.hist(train_dataset.target,bins=20, alpha=0.5, histtype='bar', ec='black',
#        label=categories)

In [3]:
import nltk
from nltk.corpus import stopwords
#nltk.download('wordnet')
#nltk.download('stopwords')
# nltk.download('punkt')#, if you need "tokenizers/punkt/english.pickle", choose it
# nltk.download('averaged_perceptron_tagger')


In [4]:
from nltk import pos_tag
wnl = nltk.wordnet.WordNetLemmatizer()

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n'

def lemmatize_sent(list_word):
    # Text input is string, returns array of lowercased strings(words).
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(list_word)]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

def stem_lemma(doc):
    return (word for word in lemmatize_sent(analyzer(doc)) if not word.isdigit())

def stem_nolemma(doc):
    return (word for word in analyzer(doc) if not word.isdigit())

In [6]:
vectorizer_df3_lemma = CountVectorizer(min_df=3, 
                             analyzer=stem_lemma, 
                             stop_words='english')
vectorizer_df5_lemma = CountVectorizer(min_df=5, 
                             analyzer=stem_lemma, 
                             stop_words='english')
vectorizer_df3_nolemma = CountVectorizer(min_df=3, 
                             analyzer=stem_nolemma, 
                             stop_words='english')
vectorizer_df5_nolemma = CountVectorizer(min_df=5, 
                             analyzer=stem_nolemma, 
                             stop_words='english')


In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.base import BaseEstimator, TransformerMixin
class SparseToDenseArray(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def transform(self, X, *_):
        if hasattr(X, 'toarray'):
            return X.toarray()
        return X

    def fit(self, *_):
        return self

In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV

class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

## Question 8

### Part 1: Keep "headers" and "footers"

#### Classify the documents into two classes

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = ['comp.graphics', 'comp.os.ms-windows.misc',
'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
'rec.autos', 'rec.motorcycles',
'rec.sport.baseball', 'rec.sport.hockey']
train_dataset = fetch_20newsgroups(subset = 'train', categories = categories
, shuffle = True, random_state = 42)
test_dataset = fetch_20newsgroups(subset = 'test', categories = categories
, shuffle = True, random_state = 42)

In [ ]:
_, idx = np.unique(train_dataset.target, return_index=True)
for i in range(len(idx)):
    j = np.sort(idx)[i]
    #print(train_dataset.target_names[i],': target index = ', train_dataset.target[j])

#print(train_dataset.target[15:25])
#print(train_dataset.target_names[15:25])

In [ ]:
train_dataset.bintarget = np.zeros_like(train_dataset.target)
test_dataset.bintarget = np.zeros_like(test_dataset.target)
for y in [6, 7, 4, 2]:
    train_dataset.bintarget[np.argwhere(train_dataset.target == y)] = 0
    test_dataset.bintarget[np.argwhere(test_dataset.target == y)] = 0
for y in [1, 3, 0, 5]:
    train_dataset.bintarget[np.argwhere(train_dataset.target == y)] = 1
    test_dataset.bintarget[np.argwhere(test_dataset.target == y)] = 1
class_names = ['Computer Technology', 'Recreational Activity']
#print(train_dataset.bintarget[:20])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

# used to cache results
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.externals.joblib import Memory
# print(__doc__)
location = mkdtemp()
memory = Memory(location=location, verbose=10)

pipeline1 = Pipeline([
    ('vect', vectorizer_df3_lemma),
    ('tfidf', TfidfTransformer()),
    ('reduce_dim', TruncatedSVD(n_components=50, random_state=42)),
    ('toarr', SparseToDenseArray()),
    ('clf', SVC(kernel='linear', C=33.0)),
],
memory=memory
)

param_grid1 = [
    {
        'vect': [vectorizer_df3_lemma, 
                 vectorizer_df3_nolemma, 
                 vectorizer_df5_lemma, 
                 vectorizer_df5_nolemma],
        'reduce_dim': [TruncatedSVD(n_components=50, random_state=42),
                       NMF(n_components=50, init='random', random_state=42)],
        'clf': [SVC(kernel='linear', C=33.0), 
                LogisticRegression(penalty='l1', C=3.3), 
                LogisticRegression(penalty='l2', C=3.3),
                GaussianNB()]
    }
]
grid1 = GridSearchCV(pipeline1, cv=5, n_jobs=1, param_grid=param_grid1, scoring='accuracy')
grid1.fit(train_dataset.data, train_dataset.bintarget)
rmtree(location)

In [ ]:
pd.DataFrame(grid1.cv_results_)

### Part 2: Remove "headers" and "footers"

#### Classify the documents into two classes

In [9]:
from sklearn.datasets import fetch_20newsgroups
categories = ['comp.graphics', 'comp.os.ms-windows.misc',
'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
'rec.autos', 'rec.motorcycles',
'rec.sport.baseball', 'rec.sport.hockey']
train_dataset = fetch_20newsgroups(subset = 'train', categories = categories
, shuffle = True, random_state = 42, remove = ('headers', 'footers'))
test_dataset = fetch_20newsgroups(subset = 'test', categories = categories
, shuffle = True, random_state = 42, remove = ('headers', 'footers'))

In [10]:
_, idx = np.unique(train_dataset.target, return_index=True)
for i in range(len(idx)):
    j = np.sort(idx)[i]
    #print(train_dataset.target_names[i],': target index = ', train_dataset.target[j])

#print(train_dataset.target[15:25])
#print(train_dataset.target_names[15:25])

In [11]:
train_dataset.bintarget = np.zeros_like(train_dataset.target)
test_dataset.bintarget = np.zeros_like(test_dataset.target)
for y in [6, 7, 4, 2]:
    train_dataset.bintarget[np.argwhere(train_dataset.target == y)] = 0
    test_dataset.bintarget[np.argwhere(test_dataset.target == y)] = 0
for y in [1, 3, 0, 5]:
    train_dataset.bintarget[np.argwhere(train_dataset.target == y)] = 1
    test_dataset.bintarget[np.argwhere(test_dataset.target == y)] = 1
class_names = ['Computer Technology', 'Recreational Activity']
#print(train_dataset.bintarget[:20])

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

# used to cache results
from tempfile import mkdtemp
from shutil import rmtree
from sklearn.externals.joblib import Memory
# print(__doc__)
location = mkdtemp()
memory = Memory(location=location, verbose=10)

pipeline2 = Pipeline([
    ('vect', vectorizer_df3_lemma),
    ('tfidf', TfidfTransformer()),
    ('reduce_dim', TruncatedSVD(n_components=50, random_state=42)),
    ('toarr', SparseToDenseArray()),
    ('clf', SVC(kernel='linear', C=33.0)),
],
memory=memory
)

param_grid2 = [
    {
        'vect': [vectorizer_df3_lemma, 
                 vectorizer_df3_nolemma, 
                 vectorizer_df5_lemma, 
                 vectorizer_df5_nolemma],
        'reduce_dim': [TruncatedSVD(n_components=50, random_state=42),
                       NMF(n_components=50, init='random', random_state=42)],
        'clf': [SVC(kernel='linear', C=33.0), 
                LogisticRegression(penalty='l1', C=3.3), 
                LogisticRegression(penalty='l2', C=3.3),
                GaussianNB()]
    }
]
grid2 = GridSearchCV(pipeline2, cv=5, n_jobs=1, param_grid=param_grid2, scoring='accuracy')
grid2.fit(train_dataset.data, train_dataset.bintarget)
rmtree(location)

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(CountVectorizer(analyzer=<function stem_lemma at 0x00000245E8EA8C80>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=3, ngram_range=(1, 1), preprocessor=None,
        stop_words='english', strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None), 
[ 'mars@ixos.de (Martin Stein) writes:\n'
  '\n'
  '#I use xwd/xpr (from the X11R5 dist.) and various programs of the\n'
  '#ppm-tools to print hardcopies of colored X windows. My problem is,\n'
  '\n'
  "I don't like xpr. It gives (at least, the X11R4 version does) louzy\n"
  'output: the hardcopy looks very grainy to me.\n'
  'Instead, I use pnmtops. This takes full advantage PostScript, and\n'
  'lets the printer do th

F:\jupyter\lib\site-packages\sklearn\pipeline.py:230: UserWarning: Persisting input arguments took 2.06s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])


_______________________________________________fit_transform_one - 63.5s, 1.1min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True), <3784x12017 sparse matrix of type '<class 'numpy.int64'>'
	with 339111 stored elements in Compressed Sparse Row format>, 
array([1, ..., 0], dtype=int64), None)
________________________________________________fit_transform_one - 0.1s, 0.0min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=42, tol=0.0), 
<3784x12017 sparse matrix of type '<class 'numpy.float64'>'
	with 339111 stored elements in Compressed Sparse Row format>, 
array([1, ..., 0], dtype=int64), None)
_______________________

F:\jupyter\lib\site-packages\sklearn\pipeline.py:230: UserWarning: Persisting input arguments took 1.72s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])


_______________________________________________fit_transform_one - 62.2s, 1.0min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True), <3786x11638 sparse matrix of type '<class 'numpy.int64'>'
	with 332706 stored elements in Compressed Sparse Row format>, 
array([0, ..., 0], dtype=int64), None)
________________________________________________fit_transform_one - 0.0s, 0.0min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=42, tol=0.0), 
<3786x11638 sparse matrix of type '<class 'numpy.float64'>'
	with 332706 stored elements in Compressed Sparse Row format>, 
array([0, ..., 0], dtype=int64), None)
_______________________

...

[Memory]0.0s, 0.0min    : Loading _fit_transform_one from C:\Users\yijui\AppData\Local\Temp\tmpuosas4ye\joblib\sklearn\pipeline\_fit_transform_one\b4edc87a2c44a8700f96761d70cf0e39
___________________________________fit_transform_one cache loaded - 0.2s, 0.0min
[Memory]0.3s, 0.0min    : Loading _fit_transform_one from C:\Users\yijui\AppData\Local\Temp\tmpuosas4ye\joblib\sklearn\pipeline\_fit_transform_one\2e737a16a686787a61f1faae38987731
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.3s, 0.0min    : Loading _fit_transform_one from C:\Users\yijui\AppData\Local\Temp\tmpuosas4ye\joblib\sklearn\pipeline\_fit_transform_one\4afcbe08d735be3a8707b594ab64eb30
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.3s, 0.0min    : Loading _fit_transform_one from C:\Users\yijui\AppData\Local\Temp\tmpuosas4ye\joblib\sklearn\pipeline\_fit_transform_one\71f1ce6e385a3ddfc8a7c12115fadf08
___________________________________fi

[Memory]0.0s, 0.0min    : Loading _fit_transform_one from C:\Users\yijui\AppData\Local\Temp\tmpuosas4ye\joblib\sklearn\pipeline\_fit_transform_one\33ad9a61f27d2fa82e1316ca858a1ecd
___________________________________fit_transform_one cache loaded - 0.1s, 0.0min
[Memory]0.2s, 0.0min    : Loading _fit_transform_one from C:\Users\yijui\AppData\Local\Temp\tmpuosas4ye\joblib\sklearn\pipeline\_fit_transform_one\86d36e63e9ac3168246af318bbe8d691
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.2s, 0.0min    : Loading _fit_transform_one from C:\Users\yijui\AppData\Local\Temp\tmpuosas4ye\joblib\sklearn\pipeline\_fit_transform_one\fed87ebc8b0d610afe7554b28bd50266
___________________________________fit_transform_one cache loaded - 0.0s, 0.0min
[Memory]0.2s, 0.0min    : Loading _fit_transform_one from C:\Users\yijui\AppData\Local\Temp\tmpuosas4ye\joblib\sklearn\pipeline\_fit_transform_one\ac54fd4a849141012ca218ee58cb1dab
___________________________________fi

________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(CountVectorizer(analyzer=<function stem_lemma at 0x00000245E8EA8C80>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=5, ngram_range=(1, 1), preprocessor=None,
        stop_words='english', strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None), 
[ 'In article <mssC5KCru.5Ip@netcom.com>, mss@netcom.com (Mark Singer) '
  'writes:\n'
  '|> \n'
  '|> \n'
  '|> The media is beating the incident at Dodger Stadium on Wednesday to\n'
  "|> death, but I haven't seen anything in rsb yet.\n"
  '|> \n'
  '|> Gerald Perry of the Cardinals pinch hit in the eighth inning with two\n'
  '|> on and his club down by a run.  He stroked a line drive into the\n'
  '|> right field corn

F:\jupyter\lib\site-packages\sklearn\pipeline.py:230: UserWarning: Persisting input arguments took 1.31s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  **fit_params_steps[name])


_______________________________________________fit_transform_one - 66.0s, 1.1min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True), <4732x8820 sparse matrix of type '<class 'numpy.int64'>'
	with 408983 stored elements in Compressed Sparse Row format>, 
array([0, ..., 0], dtype=int64), None)
________________________________________________fit_transform_one - 0.0s, 0.0min
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=42, tol=0.0), 
<4732x8820 sparse matrix of type '<class 'numpy.float64'>'
	with 408983 stored elements in Compressed Sparse Row format>, 
array([0, ..., 0], dtype=int64), None)
_________________________

In [13]:
pd.DataFrame(grid2.cv_results_)

F:\jupyter\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\jupyter\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\jupyter\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
F:\jupyter\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_reduce_dim,param_vect,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,60.814901,5.838998,14.164107,1.225053,"SVC(C=33.0, cache_size=200, class_weight=None,...","TruncatedSVD(algorithm='randomized', n_compone...",CountVectorizer(analyzer=<function stem_lemma ...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.867089,0.867865,...,0.866019,0.006186,3,0.876586,0.866614,0.874010,0.870576,0.874538,0.872465,0.003506
1,5.065603,0.095880,0.251627,0.007453,"SVC(C=33.0, cache_size=200, class_weight=None,...","TruncatedSVD(algorithm='randomized', n_compone...",CountVectorizer(analyzer=<function stem_nolemm...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.858650,0.872093,...,0.864751,0.013087,4,0.881342,0.857369,0.867670,0.866614,0.868199,0.868239,0.007653
2,54.999849,0.663563,13.351190,0.916369,"SVC(C=33.0, cache_size=200, class_weight=None,...","TruncatedSVD(algorithm='randomized', n_compone...",CountVectorizer(analyzer=<function stem_lemma ...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.866034,0.875264,...,0.867075,0.009337,1,0.876057,0.869519,0.872953,0.870576,0.871896,0.872200,0.002252
3,4.208587,0.042251,0.256537,0.013124,"SVC(C=33.0, cache_size=200, class_weight=None,...","TruncatedSVD(algorithm='randomized', n_compone...",CountVectorizer(analyzer=<function stem_nolemm...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.857595,0.866808,...,0.863271,0.012272,8,0.876321,0.859746,0.867670,0.869255,0.870312,0.868661,0.005333
4,18.179414,1.622625,13.337265,0.694429,"SVC(C=33.0, cache_size=200, class_weight=None,...","NMF(alpha=0.0, beta_loss='frobenius', init='ra...",CountVectorizer(analyzer=<function stem_lemma ...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.848101,0.858351,...,0.857143,0.007287,15,0.857030,0.863444,0.870048,0.861067,0.869255,0.864169,0.004930
5,19.246075,1.288426,0.349848,0.031618,"SVC(C=33.0, cache_size=200, class_weight=None,...","NMF(alpha=0.0, beta_loss='frobenius', init='ra...",CountVectorizer(analyzer=<function stem_nolemm...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.836498,0.843552,...,0.847422,0.008436,20,0.853858,0.851030,0.858162,0.858162,0.853407,0.854924,0.002813
6,13.151000,0.339772,13.308975,0.907592,"SVC(C=33.0, cache_size=200, class_weight=None,...","NMF(alpha=0.0, beta_loss='frobenius', init='ra...",CountVectorizer(analyzer=<function stem_lemma ...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.855485,0.859408,...,0.857988,0.002400,14,0.860465,0.863708,0.858426,0.856841,0.870840,0.862056,0.004956
7,14.948525,0.189283,0.338494,0.013059,"SVC(C=33.0, cache_size=200, class_weight=None,...","NMF(alpha=0.0, beta_loss='frobenius', init='ra...",CountVectorizer(analyzer=<function stem_nolemm...,"{'clf': SVC(C=33.0, cache_size=200, class_weig...",0.837553,0.860465,...,0.851014,0.008076,19,0.854387,0.856313,0.860275,0.848125,0.861331,0.856086,0.004720
8,0.904084,0.885454,13.361862,1.193603,"LogisticRegression(C=3.3, class_weight=None, d...","TruncatedSVD(algorithm='randomized', n_compone...",CountVectorizer(analyzer=<function stem_lemma ...,"{'clf': LogisticRegression(C=3.3, class_weight...",0.860759,0.864693,...,0.864117,0.009230,5,0.873414,0.863444,0.869255,0.866086,0.871104,0.868661,0.003543
9,0.453820,0.013755,0.206338,0.009236,"LogisticRegression(C=3.3, class_weight=None, d...","TruncatedSVD(algorithm='randomized', n_compone...",CountVectorizer(analyzer=<function stem_nolemm...,"{'clf': LogisticRegression(C=3.3, class_weight...",0.860759,0.869979,...,0.863905,0.012407,6,0.875793,0.856577,0.867406,0.866878,0.868199,0.866971,0.006125
